# Introduction to UNIQ for DNAC
## Overview

This notebook shows how to use the uniq python library with DNAC.



## Getting Started
The first requirement is to import a NbClientManager to connect to DNAC

In [1]:
import json
import requests.exceptions

# this is the client manager used to connect to DNAC
from uniq.apis.nb.client_manager import NbClientManager

# this will be the ip/dns name of DNAC, the user/password credentials
# stored in current directory
from dnac_config import DNAC, DNAC_USER, DNAC_PASSWORD

Next we create a connection, and check for obvious errors.


In [2]:
# create the connection to DNAC
try:
    dnac = NbClientManager(
                server=DNAC,
                username=DNAC_USER,
                password=DNAC_PASSWORD,
                connect=True)

except requests.exceptions.HTTPError as exc_info:
    if exc_info.response.status_code == 401:
        print('Authentication Failed. Please provide valid username/password.')
    else:
        print('HTTP Status Code {code}. Reason: {reason}'.format(
                    code=exc_info.response.status_code,
                    reason=exc_info.response.reason))
    exit(1)
except requests.exceptions.ConnectionError:
    print('Connection aborted. Please check if the host {host} is available.'.format(host=DNAC))
    exit(1)

https://sandboxdnac.cisco.com/api/system/v1/auth/token


If you print the connection, you will see the server/username/password and version of the API

In [4]:
print(dnac)

[APIC: <server:sandboxdnac.cisco.com> <username:devnetuser> <password:Cisco123!> <version:v1>]


## First API Request
This example gets all of the network devices from the controller inventory.  

### Accessing network-devices on DNAC

In [5]:
# this method will get all network devices
network_devices  = dnac.networkdevice.getAllNetworkDevice()

https://sandboxdnac.cisco.com/api/v1/network-device


In [6]:
# now pretty print it
# serialize turns object into a json dict.
# we do not normally do this, but shows the structure maps to a dictionary
print(json.dumps(dnac.serialize(network_devices), indent=2))

{
  "version": "1.0",
  "response": [
    {
      "macAddress": "00:c8:8b:80:bb:00",
      "type": "Cisco ASR 1001-X Router",
      "inventoryStatusDetail": "<status><general code=\"SUCCESS\"/></status>",
      "role": "BORDER ROUTER",
      "tagCount": "0",
      "lineCardCount": "9",
      "interfaceCount": "10",
      "roleSource": "AUTO",
      "lastUpdateTime": 1516596743686,
      "softwareVersion": "16.6.1",
      "collectionStatus": "Managed",
      "serialNumber": "FXS1932Q1SE",
      "family": "Routers",
      "lastUpdated": "2018-01-22 04:52:23",
      "series": "Cisco ASR 1000 Series Aggregation Services Routers",
      "managementIpAddress": "10.10.22.74",
      "bootDateTime": "2018-01-11 15:47:04",
      "apManagerInterfaceIp": "",
      "memorySize": "3956371104",
      "platformId": "ASR1001-X",
      "lineCardId": "a2406c7a-d92a-4fe6-b3d5-ec6475be8477, 5b75b5fd-21e3-4deb-a8f6-6094ff73e2c8, 8768c6f1-e19b-4c62-a4be-51c001b05b0f, afdfa337-bd9c-4eb0-ae41-b7a97f5f473d, c59

### Displaying data

Now print out a subset of the attributes.  Define a format string that can be used for both the heading and the data.

- UseCase: Print out a comma sepparated list of attributes for importing into an asset management system 

In [8]:
# define a formating string
formatstring='{ip:<16s}, {name:<30s}, {software:<16s}, {serial:<20s}'


# print a heading
print(formatstring.format(ip="IP Address", 
                          name="Device Name", 
                          software="Software Version",
                         serial="Serial Number"))

# print each of the nework devices.  network_devices is a list of objects with attributes, not a python dict
for network_device in network_devices.response:
    print(formatstring.format(ip=network_device.managementIpAddress, 
                                 name=network_device.hostname,
                                 software=network_device.softwareVersion,
                             serial=network_device.serialNumber))

IP Address      , Device Name                   , Software Version, Serial Number       
10.10.22.74     , asr1001-x.abc.inc             , 16.6.1          , FXS1932Q1SE         
10.10.22.66     , cat_9k_1.abc.inc              , 16.6.1          , FCW2136L0AK         
10.10.22.70     , cat_9k_2.abc.inc              , 16.6.1          , FCW2140L039         
10.10.22.69     , cs3850.abc.inc                , 16.6.2s         , FOC1833X0AR         


<font color=blue>
<hr>
<h2> Challenge </h2>
Either insert a cell or modify the cell above to change the attributes that are printed out.
<hr>
</font>

## Getting a Specific resource

Now find a specific device and make a request for a single device


In [11]:
[(net_dev.id, net_dev.upTime) for net_dev in network_devices.response ]

[('d5bbb4a9-a14d-4347-9546-89286e9f30d4', '10 days, 13:04:57.02'),
 ('6d3eaa5d-bb39-4cc4-8881-4a2b2668d2dc', '10 days, 14:05:15.95'),
 ('74b69532-5dc3-45a1-a0dd-6d1d10051f27', '10 days, 14:18:30.48'),
 ('8be78ab1-d684-49c1-8529-2b08e9c5a6d4', '7 days, 1:05:06.02')]

In [17]:
# look for a specific access device which is a 9300
try:
    # one line list comprehension
    deviceId = [net_dev.id for net_dev in network_devices.response 
                 if net_dev.role =="ACCESS" and  "9300" in net_dev.type ][0]
    print ("Success, found id:{id}".format(id=deviceId))
except keyError:
    print ("FAIL: no suitable device found")

Success, found id:6d3eaa5d-bb39-4cc4-8881-4a2b2668d2dc


This id can be used to return all the information about a single device.  Device id are important as the uniquely identify the device.  They are used in many API calls to perform an action on a device.  For example to assign a tag or a location for a device, you would need to know the ```id```.

In [20]:
network_device = dnac.networkdevice.getNetworkDeviceById(id=deviceId)
print (json.dumps(dnac.serialize(network_device), indent=2))

https://sandboxdnac.cisco.com/api/v1/network-device/6d3eaa5d-bb39-4cc4-8881-4a2b2668d2dc
{
  "version": "1.0",
  "response": {
    "macAddress": "f8:7b:20:67:62:80",
    "type": "Cisco Catalyst 9300 Switch",
    "inventoryStatusDetail": "<status><general code=\"SUCCESS\"/></status>",
    "role": "ACCESS",
    "tagCount": "0",
    "lineCardCount": "2",
    "interfaceCount": "41",
    "roleSource": "AUTO",
    "lastUpdateTime": 1516596460635,
    "softwareVersion": "16.6.1",
    "collectionStatus": "Managed",
    "serialNumber": "FCW2136L0AK",
    "family": "Switches and Hubs",
    "lastUpdated": "2018-01-22 04:47:40",
    "series": "Cisco Catalyst 9300 Series Switches",
    "managementIpAddress": "10.10.22.66",
    "bootDateTime": "2018-01-11 14:42:33",
    "apManagerInterfaceIp": "",
    "memorySize": "889225280",
    "platformId": "C9300-24UX",
    "lineCardId": "feb42c9f-323f-4e17-87d3-c2ea924320cb, 0f0c473e-b2e0-4dcf-af11-9e7cf7216473",
    "snmpLocation": "",
    "instanceUuid": "6

### Get Network-device by management IP address
Looking a device up by id is challenging as you may not always know the id.  It is often the case you know the management IP address (or serialnumber) and want to get the id of the device, or some other attributes (e.g. version of code).

- UseCase: a function to allow the user to enter the IP address of a network-device and perform some other action based on the id.

In [22]:
# looks up a network device by ipaddress, and returns the id attribute
def ipToDeviceId(ipAddress):
    ip_network_device = dnac.networkdevice.getNetworkDeviceByIp(ipAddress=ipAddress)
    return ip_network_device.response.id

# we are cheating here as we are just picking the IP address from earlier
# this is just to illustrate the use of a known ip address
ipAddress=network_device.response.managementIpAddress
id1 = ipToDeviceId(ipAddress)
print ("Example1: %s" % id1)

# now use a specific IP address.  You can change this to one of the examples above
ipAddress="10.10.22.70"
id2 = ipToDeviceId(ipAddress)
print ("Example1: %s" % id2)

https://sandboxdnac.cisco.com/api/v1/network-device/ip-address/10.10.22.66
Example1: 6d3eaa5d-bb39-4cc4-8881-4a2b2668d2dc
https://sandboxdnac.cisco.com/api/v1/network-device/ip-address/10.10.22.70
Example1: 74b69532-5dc3-45a1-a0dd-6d1d10051f27


<hr>
<font color=blue>
<h2> Challenge </h2>
Write some code to get all network-devices running IOS "15.2(4)M6a"
</font>
<hr>

## Host data store
This call show the hosts connected to the network.

- UseCase: find my host


In [24]:
hosts = dnac.host.getHosts()
print(json.dumps(dnac.serialize(hosts), indent=4))

https://sandboxdnac.cisco.com/api/v1/host
{
    "version": "1.0",
    "response": [
        {
            "lastUpdated": "1516597275026",
            "hostMac": "00:1e:13:a5:b9:40",
            "vlanId": "1",
            "connectedNetworkDeviceIpAddress": "10.10.22.70",
            "hostType": "wired",
            "subType": "UNKNOWN",
            "hostIp": "10.10.22.114",
            "connectedNetworkDeviceId": "74b69532-5dc3-45a1-a0dd-6d1d10051f27",
            "connectedInterfaceName": "TenGigabitEthernet1/0/24",
            "connectedInterfaceId": "c39338e4-93fc-4fcb-8de5-bbbd2860d950",
            "id": "ac385e41-165b-4de7-a49b-42ba15cacd4f",
            "source": "200"
        },
        {
            "lastUpdated": "1516597965806",
            "hostMac": "c8:4c:75:68:b2:c0",
            "vlanId": "1",
            "connectedNetworkDeviceIpAddress": "10.10.22.66",
            "hostType": "wired",
            "subType": "UNKNOWN",
            "hostIp": "10.10.22.98",
            "c

Lookup a device by ipaddress (macAddress, hostname).  
Can also use connectedInterfaceName and connectedNetworkDeviceIpAddress

In [25]:
# lookup host by IP address
host=dnac.host.getHosts(hostIp="10.10.22.114").response[0]

print("host: {ip} {mac}: DEVICE{deviceIp} {interface}".format(
                        ip=host.hostIp,
                        mac=host.hostMac,
                        deviceIp=host.connectedNetworkDeviceIpAddress,
                        interface=host.connectedInterfaceName   ))

https://sandboxdnac.cisco.com/api/v1/host


IndexError: list index out of range

<hr>
<font color=blue>
<h2> Challenge </h2>
Write some code to get the IOS version of the device a host is connected to.
</font>
<hr>

### interfaces

Look at the interfaces that the device above is connected to.  Get switch port utilization


In [35]:
host=dnac.host.getHosts(hostIp="10.10.22.114").response[0]
deviceId =host.connectedNetworkDeviceId
interfaces = dnac.interface.getInterfaceByDeviceId(deviceId=deviceId).response

https://sandboxdnac.cisco.com/api/v1/host
https://sandboxdnac.cisco.com/api/v1/interface/network-device/74b69532-5dc3-45a1-a0dd-6d1d10051f27


In [36]:
print (json.dumps(dnac.serialize(interfaces), indent=4))

[
    {
        "portType": "Ethernet Port",
        "portMode": "dynamic_auto",
        "series": "Cisco Catalyst 9300 Series Switches",
        "macAddress": "f8:7b:20:71:4d:a5",
        "instanceUuid": "8c8b425f-b656-4f22-9f66-c95aa41fc013",
        "deviceId": "74b69532-5dc3-45a1-a0dd-6d1d10051f27",
        "adminStatus": "UP",
        "interfaceType": "Physical",
        "portName": "FortyGigabitEthernet1/1/1",
        "status": "down",
        "lastUpdated": "2018-01-22 05:01:29.324",
        "duplex": "AutoNegotiate",
        "serialNo": "FCW2140L039",
        "pid": "C9300-24UX",
        "vlanId": "",
        "description": "",
        "ospfSupport": "false",
        "className": "EthrntPrtclEndpntExtndd",
        "id": "8c8b425f-b656-4f22-9f66-c95aa41fc013",
        "isisSupport": "false",
        "ifIndex": "44",
        "speed": "40000000"
    },
    {
        "portType": "Ethernet Port",
        "portMode": "dynamic_auto",
        "series": "Cisco Catalyst 9300 Series Switc

In [38]:
# get a list of portName,  speed and status
[(port.portName, port.speed, port.status)
                 for port in interfaces
                 if port.interfaceType == 'Physical']

[('FortyGigabitEthernet1/1/1', '40000000', 'down'),
 ('FortyGigabitEthernet1/1/2', '40000000', 'down'),
 ('GigabitEthernet0/0', '1000000', 'down'),
 ('GigabitEthernet1/1/1', '1000000', 'down'),
 ('GigabitEthernet1/1/2', '1000000', 'down'),
 ('GigabitEthernet1/1/3', '1000000', 'down'),
 ('GigabitEthernet1/1/4', '1000000', 'down'),
 ('TenGigabitEthernet1/0/1', '1000000', 'down'),
 ('TenGigabitEthernet1/0/2', '10000000', 'down'),
 ('TenGigabitEthernet1/0/3', '10000000', 'down'),
 ('TenGigabitEthernet1/0/4', '10000000', 'down'),
 ('TenGigabitEthernet1/0/5', '10000000', 'down'),
 ('TenGigabitEthernet1/0/6', '10000000', 'down'),
 ('TenGigabitEthernet1/0/7', '10000000', 'down'),
 ('TenGigabitEthernet1/0/8', '10000000', 'down'),
 ('TenGigabitEthernet1/0/9', '10000000', 'down'),
 ('TenGigabitEthernet1/0/10', '10000000', 'down'),
 ('TenGigabitEthernet1/0/11', '10000000', 'down'),
 ('TenGigabitEthernet1/0/12', '10000000', 'down'),
 ('TenGigabitEthernet1/0/13', '10000000', 'down'),
 ('TenGigabitEt

In [39]:
import re
def atoi(text):
    return int(text) if text.isdigit() else text

# natural sort for interfaces
def natural_sort(interfacelist):
    return sorted(interfacelist, key=lambda port: [ atoi(c) for c in re.split('(\d+)', port.portName)])

[(port.portName, port.speed, port.status)
        for port in natural_sort(interfaces)
            if port.interfaceType == 'Physical']

[('FortyGigabitEthernet1/1/1', '40000000', 'down'),
 ('FortyGigabitEthernet1/1/2', '40000000', 'down'),
 ('GigabitEthernet0/0', '1000000', 'down'),
 ('GigabitEthernet1/1/1', '1000000', 'down'),
 ('GigabitEthernet1/1/2', '1000000', 'down'),
 ('GigabitEthernet1/1/3', '1000000', 'down'),
 ('GigabitEthernet1/1/4', '1000000', 'down'),
 ('TenGigabitEthernet1/0/1', '1000000', 'down'),
 ('TenGigabitEthernet1/0/2', '10000000', 'down'),
 ('TenGigabitEthernet1/0/3', '10000000', 'down'),
 ('TenGigabitEthernet1/0/4', '10000000', 'down'),
 ('TenGigabitEthernet1/0/5', '10000000', 'down'),
 ('TenGigabitEthernet1/0/6', '10000000', 'down'),
 ('TenGigabitEthernet1/0/7', '10000000', 'down'),
 ('TenGigabitEthernet1/0/8', '10000000', 'down'),
 ('TenGigabitEthernet1/0/9', '10000000', 'down'),
 ('TenGigabitEthernet1/0/10', '10000000', 'down'),
 ('TenGigabitEthernet1/0/11', '10000000', 'down'),
 ('TenGigabitEthernet1/0/12', '10000000', 'down'),
 ('TenGigabitEthernet1/0/13', '10000000', 'down'),
 ('TenGigabitEt

In [40]:
len([port.portName for port in natural_sort(interfaces)
                 if port.interfaceType == 'Physical'
                    and port.status == 'up'])

2

<hr>
<font color=blue>
<h2> Challenge </h2>
Calculate the percentage utilization (active ports) for the switch above.
</font>
<hr>